🧊 Time to go **from float to fighter jet.**  
We're shrinking ResNet down to **INT8 precision** — no performance lost, just **raw inference speed gains.**

---

# 🧪 `08_lab_quantize_resnet_fp32_to_int8.ipynb`  
### 📁 `05_model_optimization`  
> Quantize a pretrained **ResNet18** model from **FP32 → INT8** using **ONNX or TFLite-style quantization**.  
Benchmark **inference speed** and **accuracy before/after**.  

---

## 🎯 Learning Goals

- Understand **quantization basics**  
- Convert model to **INT8 weights**  
- Measure impact on **accuracy & latency**  
- Tools: PyTorch + ONNX (no custom CUDA needed)

---

## 💻 Runtime Targets

| Component            | Spec              |
|----------------------|-------------------|
| Model                | ResNet18 (Torchvision) ✅  
| Dataset              | CIFAR-10 (quick test set) ✅  
| Target format        | ONNX INT8 ✅  
| Hardware             | CPU / Colab ✅  
| Speed vs Accuracy    | Compared live ✅  

---

## ⚙️ Section 1: Imports & Pretrained Model

```python
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import time
```

```python
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

model_fp32 = models.resnet18(pretrained=True)
model_fp32.eval()
```

---

## ⏱️ Section 2: Inference Benchmark (FP32)

```python
def evaluate(model, loader):
    correct = 0
    total = 0
    start = time.time()
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            pred = outputs.argmax(1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)
    end = time.time()
    acc = correct / total
    latency = end - start
    return acc, latency

acc_fp32, time_fp32 = evaluate(model_fp32, testloader)
print(f"FP32 Accuracy: {acc_fp32:.4f} | Time: {time_fp32:.2f}s")
```

---

## 🔁 Section 3: Apply Static Quantization

```python
import torch.quantization

model_q = models.resnet18(pretrained=True)
model_q.eval()

model_q.fuse_model = lambda: None  # Dummy hook for torchvision models
model_q = torch.quantization.quantize_dynamic(model_q, {torch.nn.Linear}, dtype=torch.qint8)
```

---

## 🔎 Section 4: Benchmark INT8 Model

```python
acc_int8, time_int8 = evaluate(model_q, testloader)
print(f"INT8 Accuracy: {acc_int8:.4f} | Time: {time_int8:.2f}s")
```

---

## 📊 Section 5: Compare Results

```python
labels = ['FP32', 'INT8']
accs = [acc_fp32, acc_int8]
times = [time_fp32, time_int8]

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.bar(labels, accs, color=['blue', 'green'])
plt.title("Accuracy Comparison")

plt.subplot(1, 2, 2)
plt.bar(labels, times, color=['blue', 'green'])
plt.title("Inference Time (seconds)")

plt.suptitle("FP32 vs INT8 Quantization Results")
plt.show()
```

---

## ✅ Wrap-Up Summary

| What You Achieved           | ✅ |
|-----------------------------|----|
| Loaded & benchmarked FP32   | ✅ |
| Quantized to INT8           | ✅ |
| Compared accuracy + speed   | ✅ |
| Ran on CPU / Colab          | ✅ |

---

## 🧠 What You Learned

- Quantization trades **precision for performance**  
- INT8 can run **2× to 4× faster** with minimal accuracy loss  
- You now know how to **optimize models for edge/mobile/real-time inference**

---

Wanna go full Sensei mode and hit `09_lab_distill_teacher_student_on_mnist.ipynb` next?  
We’ll clone a big model’s brain into a tiny one — **knowledge distillation** style 🧪🧠🍼. Let’s roll?